In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import sys
from fxpmath import Fxp

In [2]:
dir_current = os.path.abspath('')
dir_parent  = os.path.dirname(dir_current)
if not dir_parent in sys.path: sys.path.append(dir_parent)

## Parametros de Simulacion

In [3]:
#Parametros de Cuantizacion
bits_activaciones      = 12
precision_activaciones = 6
bits_pesos_conv        = 12
precision_pesos_conv   = 8
bits_pesos_fc          = 6
precision_pesos_fc     = 6

# Parametros de buffer input/output
Malpc = 12    #Maximo de activaciones leidas en un ciclo
Maepc = 12    #Maximo de activaciones escritas en un ciclo
bits_lectura_input    = Malpc*bits_activaciones
bits_escritura_output = Maepc*bits_activaciones

#Parametros de la matriz de dsp slices.
filas      = Malpc
dsp_slices = 720
columnas   = dsp_slices/filas

#Parametros de Buffers de pesos
Mpclpc = 24   # Maximo de pesos convolucionales leidos en un ciclo
Mpflpc = 48   # Maximo de pesos full conectados leidos en un ciclo
bits_lectura_pesos_conv  = Mpclpc*bits_pesos_conv
bits_lectura_pesos_fc    = Mpflpc*bits_pesos_fc

#Tamaño del buffer
IOBuffer_size = 290400*bits_activaciones

## Carga del modelo y sus pesos

In [4]:
from models import AlexNet_body
from models import Weight_Quantization

# Creacion de la red
Nclasses = 10
Qinput_layer  = tf.keras.Input((227,227,3))
Qoutput_layer = AlexNet_body(Qinput_layer, Quantization = True, word_size = bits_activaciones, frac_size = precision_activaciones, N_labels=Nclasses)
QAlexNet = tf.keras.Model(inputs=Qinput_layer, outputs=Qoutput_layer)

# Carga de los pesos
cwd = os.getcwd()
Wgt_dir = os.path.join(cwd,'TrainedWeights')
Wgt_dir = os.path.join(Wgt_dir,'WeedWeights')
Wgt_dir = os.path.join(Wgt_dir,'Weights')
QAlexNet.load_weights(Wgt_dir)

# Cuantizacion de los pesos
Weight_Quantization(model = QAlexNet, 
                    Frac_Bits = precision_pesos_conv, Int_Bits = (bits_pesos_conv-precision_pesos_conv-1),
                    Dense_Frac_Bits = precision_pesos_fc, Dense_Int_Bits = (bits_pesos_fc-precision_pesos_fc-1))


## Carga del dataset

In [5]:
from models import Quantization_layer
from functions import AlexNet_resize_v2

x_test,  _  = tfds.as_numpy(tfds.load('deep_weeds', split='train[85%:90%]', batch_size=-1, as_supervised=True))
del _
# Normalizacion
x_test = x_test/255.
# Cuantizacion
x_test = Quantization_layer(x_test)


test_dataset = tf.data.Dataset.from_tensor_slices(x_test)
test_dataset = test_dataset.map(AlexNet_resize_v2)

## Simulacion

In [ ]:
# Bufferes
IOBuffer_1 = np.zeros(IOBuffer_size,dtype=int)
IOBuffer_2 = np.zeros(IOBuffer_size,dtype=int)

# Diccionario de estadisticas de los buffers
stats_IOBuffer_1 = {"ultimo_valor"    : np.zeros(IOBuffer_size,dtype=int),
                    "ciclos_1"        : np.zeros(IOBuffer_size,dtype=int),
                    "cambios_logicos" :np.zeros(IOBuffer_size,dtype=int)}
stats_IOBuffer_2 = {"ultimo_valor"    : np.zeros(IOBuffer_size,dtype=int),
                    "ciclos_1"        : np.zeros(IOBuffer_size,dtype=int),
                    "cambios_logicos" :np.zeros(IOBuffer_size,dtype=int)}

import simulation
import time
from functions import save_obj, load_obj, Load_Image

# Variables de simulacion
simulation.Malpc            = Malpc
simulation.Maepc            = Maepc
simulation.Mpclpc           = Mpclpc
simulation.Mpflpc           = Mpflpc
simulation.filas            = filas
simulation.columnas         = columnas
simulation.Word_bits        = bits_activaciones
simulation.Frac_bits        = precision_activaciones
simulation.QAlexNet         = QAlexNet


iterator   = iter(test_dataset)
ciclos     = 0
index      = 0
batch_size = 10
t = time.time()
while index < batch_size:
    image = next(iterator)
    if index % 2 == 0:
        ciclos += simulation.Test_Simulation2(image,IOBuffer_1,stats_IOBuffer_1,IOBuffer_2,stats_IOBuffer_2)
    else:
        ciclos += simulation.Test_Simulation2(image,IOBuffer_2,stats_IOBuffer_2,IOBuffer_1,stats_IOBuffer_1)
    save_obj(stats_IOBuffer_1,'stats_B1_Test4')
    save_obj(stats_IOBuffer_2,'stats_B2_Test4')
    index = index + 1
    print('imagenes procesadas:',index,' tiempo:',time.time()-t,'ciclos: ',ciclos)
print('Tiempo de simulacion:',time.time()-t)
save_obj(ciclos,'numero_de_ciclos_Test4')